In [1]:
import pandas as pd
import numpy as np
data = pd.read_csv("Bank_Personal_Loan_Modelling.csv")

data.head()

,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
0,1,25,1,49,91107,4,1.6,1,0,0,1,0,0,0
1,2,45,19,34,90089,3,1.5,1,0,0,1,0,0,0
2,3,39,15,11,94720,1,1.0,1,0,0,0,0,0,0
3,4,35,9,100,94112,1,2.7,2,0,0,0,0,0,0
4,5,35,8,45,91330,4,1.0,2,0,0,0,0,0,1


In [2]:
data = data.drop(columns = ["ID","ZIP Code"])

In [3]:
X = data.drop("Personal Loan",axis=1)
y = data["Personal Loan"]

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=0)

X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [4]:
import keras
from keras.models import Sequential
from keras.layers import Dense

from keras.optimizers import Adam

optimizer = Adam(lr=0.01)
loss = 'binary_crossentropy'

C:\Users\Srivatsan\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [5]:
model = Sequential()
model.add(Dense(units=6, activation='relu', input_dim=X_train.shape[1]))
model.add(Dense(units=1, activation='sigmoid'))

In [27]:
def fitness(weights):
    weights = weights.flatten()
    w0 = weights[:66].reshape((11, 6))
    w1 = weights[66:72].reshape((6,1))
    b0= np.array([0.,0.,0.,0.,0.,0.])
    b1 = np.array([0.])


    model.layers[0].set_weights([w0, b0])
    model.layers[1].set_weights([w1, b1])
    
    model.compile(optimizer=optimizer, loss=loss)
    Loss = model.evaluate(X_train, y_train, verbose=0)
    return Loss

In [28]:
def ant_colony_optimization(population_size=10, iterations=50, alpha=1, beta=2, evaporation_rate=0.5, Q=100):
    
    # initialize the pheromone matrix
    pheromone = np.ones((12, 6))
    
    
    # initialize the best weights and the loss
    best_weights = None
    best_loss = 99
    
    for i in range(iterations):
        # initialize the ant population and ant accuracies
        ants = []
        ant_loss = []
        
        for j in range(population_size):
            weights = np.zeros((12,6))
            for k in range(12):
                for l in range(6):
                    if np.random.rand() < pheromone[k][l]:
                        weights[k][l] = np.random.randn()
                        
            losses = fitness(weights)
            ants.append(weights)
            ant_loss.append(losses)
            
            
            if losses < best_loss:
                best_weights = weights
                best_loss = losses
                
                
        # update the pheromone matrix
        for k in range(11):
            for l in range(6):
                pheromone[k][l] = (1 - evaporation_rate) * pheromone[k][l]
                for j in range(population_size):
                    if ants[j][k][l] != 0:
                        pheromone[k][l] += Q / ant_loss[j] * ants[j][k][l]
       
        if i % 10 == 0:
            print("Iteration:", i, "Best Loss:", best_loss)
    
    return best_weights

In [29]:
best_weights = ant_colony_optimization()

Iteration: 0 Best Loss: 0.5727410316467285
Iteration: 10 Best Loss: 0.44059422612190247
Iteration: 20 Best Loss: 0.44059422612190247
Iteration: 30 Best Loss: 0.44059422612190247
Iteration: 40 Best Loss: 0.44059422612190247


In [31]:
best_weights = best_weights.flatten()
w0 = best_weights[:66].reshape((11, 6))
w1 = best_weights[66:72].reshape((6,1))
b0= np.array([0.,0.,0.,0.,0.,0.])
b1 = np.array([0.])


model.layers[0].set_weights([w0, b0])
model.layers[1].set_weights([w1, b1])
    
model.compile(optimizer=optimizer, loss=loss)

In [36]:
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5)

47/47 [==============================] - 0s 921us/step


In [37]:
from sklearn.metrics import accuracy_score
accuracy_score(y_pred,y_test)

0.8226666666666667